## НАШ ПРОЕКТИК

# 1. Получение данных


In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import time

# Конфигурация
API_KEY = "93a2e224b502c359fd18021ff02228f0"  # Замените на реальный ключ
CITIES = ["Moscow", "Saint Petersburg", "Nizhny Novgorod", "Kazan", "Ufa", "Novosibirsk", "Vladivostok"]

def get_city_coordinates(city, api_key):
    """Получаем координаты города с обработкой ошибок"""
    try:
        geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
        response = requests.get(geo_url)
        response.raise_for_status()  # Проверка на ошибки HTTP
        data = response.json()
        if not data:
            raise ValueError(f"Город {city} не найден")
        return data[0]["lat"], data[0]["lon"]
    except Exception as e:
        print(f"Ошибка при получении координат для {city}: {str(e)}")
        return None, None

def get_historical_weather(lat, lon, date, api_key):
    """Получаем исторические данные с обработкой ошибок"""
    try:
        url = f"https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={api_key}&units=metric"
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except Exception as e:
        print(f"Ошибка при получении данных для {lat},{lon} на {date}: {str(e)}")
        return None

# Пример использования с обработкой ошибок
try:
    # Получаем координаты Москвы
    lat, lon = get_city_coordinates("Moscow", API_KEY)
    if lat is None or lon is None:
        raise ValueError("Не удалось получить координаты Москвы")
    
    # Дата для запроса (1 января 2023)
    target_date = datetime(2023, 1, 1)
    date_unix = int(target_date.timestamp())
    
    # Получаем данные
    weather_data = get_historical_weather(lat, lon, date_unix, API_KEY)
    
    if weather_data and "hourly" in weather_data:
        print(f"Температура в Москве 1 января 2023 в 00:00: {weather_data['hourly'][0]['temp']}°C")
    else:
        print("Не удалось получить данные о погоде")
        
except Exception as e:
    print(f"Произошла ошибка: {str(e)}")

Ошибка при получении координат для Moscow: 401 Client Error: Unauthorized for url: http://api.openweathermap.org/geo/1.0/direct?q=Moscow&limit=1&appid=93a2e224b502c359fd18021ff02228f0
Произошла ошибка: Не удалось получить координаты Москвы


In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import numpy as np

# Конфигурация
API_KEY = "93a2e224b502c359fd18021ff02228f0"  # Замените на свой ключ OpenWeatherMap
CITIES = ["Moscow", "Saint Petersburg", "Nizhny Novgorod", "Kazan", "Ufa", "Novosibirsk", "Vladivostok"]
YEARS = [2023]
MONTHS = range(1, 13)
DAYS = range(1, 29)  # Упрощение: берём по 28 дней в месяце

def get_city_coordinates(city, api_key):
    """Получаем координаты города."""
    geo_url = f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={api_key}"
    response = requests.get(geo_url).json()
    return response[0]["lat"], response[0]["lon"]

def fetch_historical_weather(city, date, api_key):
    """Загружаем исторические данные по API."""
    lat, lon = get_city_coordinates(city, api_key)
    url = f"https://api.openweathermap.org/data/2.5/onecall/timemachine?lat={lat}&lon={lon}&dt={date}&appid={api_key}&units=metric"
    return requests.get(url).json()

# Пример: загрузка данных для Москвы 1 января 2023
date_unix = int(datetime(2023, 1, 1).timestamp())
weather_data = fetch_historical_weather("Moscow", date_unix, API_KEY)
print(weather_data["hourly"][0]["temp"])  # Первое значение температуры

# 2. Предобработка данных 

# 3. Анализ корреляции

3.1 Рассчет корреляций

3.2 Анализ корреляций 

# 4. Построение сетевых моделей 

4.1. Создание графов

4.2. Выделение клик и независимых множеств

# 5. Сравнение результатов